In [ ]:
import pickle
from pathlib import Path
from importlib import reload

from bikewaysim.paths import config
from bikewaysim.impedance_calibration import stochastic_optimization, loss_functions
from bikewaysim.routing import rustworkx_routing_funcs

with (config['calibration_fp']/'subsets.pkl').open('rb') as fh:
    subsets = pickle.load(fh)
subsets = [x for x in subsets if x[0] == 'debug']

kwargs = {
    'calibration_name': 'debug',    
    'betas_tup': (
        {'col':'2lpd','type':'link','range':[0,3]},
        {'col':'3+lpd','type':'link','range':[0,3]},
        {'col':'(30,inf) mph','type':'link','range':[0,3]},
        {'col':'[4k,10k) aadt','type':'link','range':[0,3]},
        {'col':'[10k,inf) aadt','type':'link','range':[0,3]},
        {'col':'[4,6) grade','type':'link','range':[0,3]},
        {'col':'[6,inf) grade','type':'link','range':[0,3]},
        {'col':'bike lane','type':'link','range':[-1,3]},
        {'col':'multi use path and cycletrack','type':'link','range':[-1,3]},
        {'col':'unsig_crossing','type':'turn','range':[0,5]},
        {'col':'left_turn','type':'turn','range':[0,5]}
    ),
    'set_to_zero': ['bike lane','cycletrack','multi use path'],
    'set_to_inf': ['not_street'],
    'objective_function': loss_functions.jaccard_buffer_total,
    'stochastic_optimization_settings': {'method':'pso','options':{'maxiter':10,'popsize':3,'return_all':True,'ftol':5}},
    'print_results': True,
    'subset': subsets[0]
}

#BUG if last results resulted in negative edge weights, then the post caliiration will fail

In [ ]:
import numpy as np
test = [np.inf,2]
np.argmin(test)

In [ ]:
reload(stochastic_optimization)
stochastic_optimization.full_impedance_calibration(**kwargs)

In [ ]:
with (config['calibration_fp']/'results/debug,debug,7.pkl').open('rb') as fh:
    test = pickle.load(fh)

In [ ]:
funall = test['results'].funall
# np.argmax(funall,axis=1)
funall

In [ ]:
test['results'].xall

In [ ]:
np.linalg.norm(xbest - x[idx]) <= xtol

In [ ]:
# figure out how to calculate xtol

In [ ]:
import numpy as np

def rosenbrock(x):
   x = np.asarray(x)
   sum1 = ((x[1:] - x[:-1] ** 2) ** 2).sum()
   sum2 = np.square(1.0 - x[:-1]).sum()
   return 100.0 * sum1 + sum2

bounds = [[-5.12, 5.12], [-5.12, 5.12]]  # The number of variables to optimize is len(bounds)

from stochopy.optimize import minimize

x = minimize(rosenbrock, bounds, method="cmaes", options={"maxiter": 100, "popsize": 10, "seed": 0, 'return_all': True, "xtol": 2,"ftol": 0})

In [ ]:
x

In [ ]:
x.xall[0]

In [ ]:
with (config['calibration_fp']/'results/all,gdot,1.pkl').open('rb') as fh:
    test = pickle.load(fh)

In [ ]:
test['results']

In [ ]:
import numpy as np
np.array(test['past_vals'])[:,-1]